In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pickle
import re

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 281Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 111Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 425Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 64.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 267Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 162Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 163Mit/s]                                                       


In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
with open("dataset.pkl", "rb") as fd:
    data = pickle.load(fd)


def preprocess(x):
    tmp = re.sub("[\[].*?[\]]", "", x.strip())
    tmp = re.sub(r'(\n){2,}','\n', tmp)
    return tmp

processed_data = []
for d in data:
    tmp = {
        "lyrics": preprocess(d["lyrics"])
    }
    processed_data.append(tmp)

file = open("data\processed_lyrics.txt", "w") 
for x in processed_data:
  file.write(x["lyrics"]) 
  file.write("\n\n")
file.close() 

In [ ]:
file_name = "data\processed_lyrics.txt"

## Finetune GPT-2

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps= 1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500,
              learning_rate = 1e-5
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


dataset has 129080 tokens
Training...
[10 | 28.92] loss=2.32 avg=2.32
[20 | 50.42] loss=1.90 avg=2.11
[30 | 72.17] loss=2.11 avg=2.11
[40 | 94.13] loss=2.08 avg=2.10
[50 | 116.24] loss=2.26 avg=2.13
[60 | 138.52] loss=2.30 avg=2.16
[70 | 160.97] loss=2.63 avg=2.23
[80 | 183.54] loss=1.64 avg=2.15
[90 | 206.25] loss=2.15 avg=2.15
[100 | 229.05] loss=2.21 avg=2.16
[110 | 251.92] loss=1.87 avg=2.13
[120 | 274.85] loss=2.03 avg=2.12
[130 | 297.82] loss=2.18 avg=2.13
[140 | 320.90] loss=2.19 avg=2.13
[150 | 343.91] loss=1.96 avg=2.12
[160 | 366.94] loss=1.83 avg=2.10
[170 | 390.03] loss=2.14 avg=2.10
[180 | 413.12] loss=1.86 avg=2.09
[190 | 436.16] loss=2.12 avg=2.09
[200 | 459.21] loss=1.84 avg=2.08
======== SAMPLE 1 ========
 you think I was wrong? You've shown me a man to be what he is in life. Now go do something else.<|endoftext|>It's a simple exercise to find yourself some space if you're going to enjoy the experience of riding in the city.
There is a difference between standing on th

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1685
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1685




Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="***SOME TITLE***",
              nsamples=5,
              batch_size=5
              )

***SOME TITLE***
I'm a homemaker, homemaker, homemaker
And the only way I know how
To get by is if I earn a wage
If you're looking for a man that's true
 don't miss it.
- 
If You're Gonna Love Me (feat. 
R.E.M.)
Arrive Early
Before I get to my appointment
Will you come early or will you wait?
Will you hurry?
Before I get to my appointment
Will you hurry?
Before I get to my appointment
Will you hurry?
O.K. 
you?
Before I got here in the morning
I had a really bad sweat condition
And my homemaker mommy worked
Would-say's'
She said, "Lay your head in the sand"
And I have so many things I want
To say but I can't really remember
Before I got here in the morning
I had a really bad sweat condition
And my mommy worked me only had one answer
Questions that she would have to answer me by
Before I got here in the morning
I had a really bad sweat condition
And my mommy worked me only had
***SOME TITLE***
So here goes no complaining, I got a job to do
Doing what I do best
I call your name, I get yo